# Quantitative Momentum Strategy

"Momentum investing" means investing in the stocks that have increased in price the most.

For this project, we're going to build an investing strategy that selects the 50 stocks with the highest price momentum. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Import

The first thing we need to do is import the open-source software libraries that we'll be using in this tutorial.

In [1]:
import numpy as np
import pandas as pd
import requests
import math
from scipy.stats import percentileofscore as score
import xlsxwriter

## Importing Our List of Stocks

As before, we'll need to import our list of stocks and our API token before proceeding. Make sure the .csv file is still in your working directory and import it with the following command:

In [2]:
stocks = pd.read_csv('sp_500_stocks.csv')
stocks = stocks[~stocks['Ticker'].isin(['DISCA', 'HFC','VIAC','WLTW','CTL','ETFC','MYL','NBL'])]
from secrets import IEX_CLOUD_API_TOKEN

## Making Our First API Call

It's now time to make the first version of our momentum screener!

We need to get one-year price returns for each stock in the universe. Here's how.

In [3]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/stats?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
data

{'companyName': 'Apple Inc',
 'marketcap': 2522676350773,
 'week52high': 187.02,
 'week52low': 129.54,
 'week52highSplitAdjustOnly': 189.46,
 'week52lowSplitAdjustOnly': 134.96,
 'week52change': -0.0523991183357556,
 'sharesOutstanding': 16678706532,
 'float': 0,
 'avg10Volume': 81795241,
 'avg30Volume': 97775325,
 'day200MovingAvg': 151.13,
 'day50MovingAvg': 151.3,
 'employees': 151387,
 'ttmEPS': 6.31,
 'ttmDividendRate': 0.946323638739966,
 'dividendYield': 0.00623469932993642,
 'nextDividendDate': '',
 'exDividendDate': '2022-10-26',
 'nextEarningsDate': '2023-01-13',
 'peRatio': 24.66378644769995,
 'beta': 1.289899673767239,
 'maxChangePercent': 58.20260655339419,
 'year5ChangePercent': 2.854748958988699,
 'year2ChangePercent': 0.2996529285714214,
 'year1ChangePercent': -0.0540848168362695,
 'ytdChangePercent': -0.1484352268490388,
 'month6ChangePercent': 0.10505101650228718,
 'month3ChangePercent': -0.11987079348079278,
 'month1ChangePercent': 0.05534112743193395,
 'day30ChangeP

## Parsing Our API Call

This API call has all the information we need. We can parse it using the same square-bracket notation as in the first project of this course. Here is an example

In [4]:
data['year1ChangePercent']

-0.0540848168362695

## Executing A Batch API Call & Building Our DataFrame

Just like in our first project, it's now time to execute several batch API calls and add the information we need to our DataFrame.

We'll start by running the following code cell, which contains some code we already built last time that we can re-use for this project. More specifically, it contains a function called chunks that we can use to divide our list of securities into groups of 100.

In [5]:
# Function sourced from 
# https://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]   

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(0, len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))
#    print(symbol_strings[i])

my_columns = ['Ticker', 'Price', 'One-Year Price Return', 'Number of Shares to Buy']

Now we need to create a blank DataFrame and add our data to the data frame one-by-one.

In [6]:
final_dataframe = pd.DataFrame(columns = my_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=quote,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        final_dataframe = final_dataframe.append(
            pd.Series(
                [symbol,
                 data[symbol]['quote']['latestPrice'],
                 data[symbol]['stats']['year1ChangePercent'],
                 'n/a'
                ],
                index = my_columns
            ),
            ignore_index = True
        )
final_dataframe   
    

,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,A,151.25,-0.109028,n/a
1,AAL,14.42,-0.281000,n/a
2,AAP,148.40,-0.348654,n/a
3,AAPL,156.47,-0.054032,n/a
4,ABBV,161.43,0.397595,n/a
...,...,...,...,...
492,YUM,130.21,-0.000261,n/a
493,ZBH,119.41,-0.112392,n/a
494,ZBRA,263.70,-0.598541,n/a
495,ZION,52.58,-0.212538,n/a


## Removing Low-Momentum Stocks

The investment strategy that we're building seeks to identify the 50 highest-momentum stocks in the S&P 500.

Because of this, the next thing we need to do is remove all the stocks in our DataFrame that fall below this momentum threshold. We'll sort the DataFrame by the stocks' one-year price return, and drop all stocks outside the top 50.

In [7]:
final_dataframe.sort_values('One-Year Price Return', ascending = False, inplace = True)
final_dataframe = final_dataframe[:50]
final_dataframe.reset_index(inplace = True)
final_dataframe

,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,271,LB,81.27,2.381720,n/a
1,351,OXY,74.75,1.483636,n/a
2,311,MPC,121.49,1.089842,n/a
3,461,VLO,143.34,1.078721,n/a
4,313,MRO,31.30,1.054363,n/a
5,109,COP,132.80,0.972943,n/a
6,215,HES,147.23,0.960267,n/a
7,488,XOM,112.09,0.948983,n/a
8,191,FTI,12.21,0.944221,n/a
9,332,NOV,23.75,0.923904,n/a


## Calculating the Number of Shares to Buy

Just like in the last project, we now need to calculate the number of shares we need to buy. The one change we're going to make is wrapping this functionality inside a function, since we'll be using it again later in this Jupyter Notebook.

In [8]:
def portfolio_input():
    global portfolio_size
    portfolio_size = input('Enter the size of your portfolio:')
    
    try:
        float(portfolio_size)
    except ValueError:
        print('That is not a number \nplease try again with only number without any sign and words')
        portfolio_size = input('Enter the size of your portfolio:')
        float(portfolio_size)

portfolio_input()
print(portfolio_size)

Enter the size of your portfolio: 100000


100000


In [9]:
position_size = float(portfolio_size)/len(final_dataframe.index)

for i in range(0, len(final_dataframe)):
    final_dataframe.loc[i, 'Number of Shares to Buy'] = math.floor(position_size/final_dataframe.loc[i, 'Price'])
final_dataframe

C:\Users\bison\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,index,Ticker,Price,One-Year Price Return,Number of Shares to Buy
0,271,LB,81.27,2.381720,24
1,351,OXY,74.75,1.483636,26
2,311,MPC,121.49,1.089842,16
3,461,VLO,143.34,1.078721,13
4,313,MRO,31.30,1.054363,63
5,109,COP,132.80,0.972943,15
6,215,HES,147.23,0.960267,13
7,488,XOM,112.09,0.948983,17
8,191,FTI,12.21,0.944221,163
9,332,NOV,23.75,0.923904,84


## Building a Better (and More Realistic) Momentum Strategy

Real-world quantitative investment firms differentiate between "high quality" and "low quality" momentum stocks:

- High-quality momentum stocks show "slow and steady" outperformance over long periods of time
- Low-quality momentum stocks might not show any momentum for a long time, and then surge upwards.

The reason why high-quality momentum stocks are preferred is because low-quality momentum can often be cause by short-term news that is unlikely to be repeated in the future (such as an FDA approval for a biotechnology company).

To identify high-quality momentum, we're going to build a strategy that selects stocks from the highest percentiles of:

- 1-month price returns
- 3-month price returns
- 6-month price returns
- 1-year price returns

Let's start by building our DataFrame. You'll notice that I use the abbreviation hqm often. It stands for high-quality momentum.

In [10]:
hqm_columns = ['Ticker',
               'Price',
               'Number of Shares to Buy',
               'One-Year Price Return',
               'One-Year Return Percentile',
               'Six-Month Price Return',
               'Six-Month Return Percentile',
               'Three-Month Price Return',
               'Three-Month Return Percentile',
               'One-Month Price Return',
               'One-Month Return Percentile',
               'HQM Score']

hqm_dataframe = pd.DataFrame(columns = hqm_columns)

for symbol_string in symbol_strings:
    batch_api_call_url = f'https://sandbox.iexapis.com/stable/stock/market/batch?symbols={symbol_string}&types=price,stats&token={IEX_CLOUD_API_TOKEN}'
    data = requests.get(batch_api_call_url).json()
    for symbol in symbol_string.split(','):
        hqm_dataframe = hqm_dataframe.append(
            pd.Series(
            [symbol,
             data[symbol]['price'],
             'N/A',
             data[symbol]['stats']['year1ChangePercent'],
             'N/A',
             data[symbol]['stats']['month6ChangePercent'],
             'N/A',
             data[symbol]['stats']['month3ChangePercent'],
             'N/A',
             data[symbol]['stats']['month1ChangePercent'],
             'N/A',
             'N/A'
            ],
            index = hqm_columns),
            ignore_index = True)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,152.94,N/A,-0.106371,N/A,0.200282,N/A,0.066431,N/A,0.135348,N/A,N/A
1,AAL,14.52,N/A,-0.275056,N/A,-0.166002,N/A,-0.008587,N/A,0.004386,N/A,N/A
2,AAP,148.10,N/A,-0.356449,N/A,-0.239451,N/A,-0.285924,N/A,-0.142189,N/A,N/A
3,AAPL,158.56,N/A,-0.054404,N/A,0.108873,N/A,-0.117072,N/A,0.055726,N/A,N/A
4,ABBV,161.56,N/A,0.401461,N/A,0.042773,N/A,0.108288,N/A,0.083550,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
492,YUM,130.66,N/A,-0.000260,N/A,0.148148,N/A,0.074096,N/A,0.129999,N/A,N/A
493,ZBH,119.44,N/A,-0.110218,N/A,-0.020279,N/A,0.010277,N/A,0.056709,N/A,N/A
494,ZBRA,256.20,N/A,-0.593489,N/A,-0.212831,N/A,-0.221915,N/A,-0.017222,N/A,N/A
495,ZION,52.73,N/A,-0.205358,N/A,-0.011802,N/A,-0.114123,N/A,0.039895,N/A,N/A


## Calculating Momentum Percentiles

We now need to calculate momentum percentile scores for every stock in the universe. More specifically, we need to calculate percentile scores for the following metrics for every stock:

- One-Year Price Return
- Six-Month Price Return
- Three-Month Price Return
- One-Month Price Return

In [11]:
time_periods = ['One-Year',
               'Six-Month',
               'Three-Month',
               'One-Month']

for row in hqm_dataframe.index:
    for time_period in time_periods:
        change_col = f'{time_period} Price Return'
        percentile_col = f'{time_period} Return Percentile'
        hqm_dataframe.loc[row, percentile_col] = score(hqm_dataframe[change_col], hqm_dataframe.loc[row, change_col])/100
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,152.94,N/A,-0.106371,0.462777,0.200282,0.83501,0.066431,0.772636,0.135348,0.645875,N/A
1,AAL,14.52,N/A,-0.275056,0.203219,-0.166002,0.120724,-0.008587,0.525151,0.004386,0.142857,N/A
2,AAP,148.10,N/A,-0.356449,0.128773,-0.239451,0.072435,-0.285924,0.026157,-0.142189,0.024145,N/A
3,AAPL,158.56,N/A,-0.054404,0.531187,0.108873,0.668008,-0.117072,0.227364,0.055726,0.273642,N/A
4,ABBV,161.56,N/A,0.401461,0.927565,0.042773,0.501006,0.108288,0.867203,0.083550,0.384306,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...
492,YUM,130.66,N/A,-0.000260,0.613682,0.148148,0.756539,0.074096,0.796781,0.129999,0.61167,N/A
493,ZBH,119.44,N/A,-0.110218,0.458753,-0.020279,0.348089,0.010277,0.607646,0.056709,0.279678,N/A
494,ZBRA,256.20,N/A,-0.593489,0.014085,-0.212831,0.090543,-0.221915,0.064386,-0.017222,0.088531,N/A
495,ZION,52.73,N/A,-0.205358,0.297787,-0.011802,0.362173,-0.114123,0.231388,0.039895,0.221328,N/A


## Calculating the HQM Score

We'll now calculate our HQM Score, which is the high-quality momentum score that we'll use to filter for stocks in this investing strategy.

The HQM Score will be the arithmetic mean of the 4 momentum percentile scores that we calculated in the last section.

To calculate arithmetic mean, we will use the mean function from Python's built-in statistics module.

In [12]:
from statistics import mean

for row in hqm_dataframe.index:
    momentum_percentiles = []
    for time_period in time_periods:
        momentum_percentiles.append(hqm_dataframe.loc[row, f'{time_period} Return Percentile'])
    hqm_dataframe.loc[row, 'HQM Score'] = mean(momentum_percentiles)

hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,A,152.94,N/A,-0.106371,0.462777,0.200282,0.83501,0.066431,0.772636,0.135348,0.645875,0.679074
1,AAL,14.52,N/A,-0.275056,0.203219,-0.166002,0.120724,-0.008587,0.525151,0.004386,0.142857,0.247988
2,AAP,148.10,N/A,-0.356449,0.128773,-0.239451,0.072435,-0.285924,0.026157,-0.142189,0.024145,0.062877
3,AAPL,158.56,N/A,-0.054404,0.531187,0.108873,0.668008,-0.117072,0.227364,0.055726,0.273642,0.42505
4,ABBV,161.56,N/A,0.401461,0.927565,0.042773,0.501006,0.108288,0.867203,0.083550,0.384306,0.67002
...,...,...,...,...,...,...,...,...,...,...,...,...
492,YUM,130.66,N/A,-0.000260,0.613682,0.148148,0.756539,0.074096,0.796781,0.129999,0.61167,0.694668
493,ZBH,119.44,N/A,-0.110218,0.458753,-0.020279,0.348089,0.010277,0.607646,0.056709,0.279678,0.423541
494,ZBRA,256.20,N/A,-0.593489,0.014085,-0.212831,0.090543,-0.221915,0.064386,-0.017222,0.088531,0.064386
495,ZION,52.73,N/A,-0.205358,0.297787,-0.011802,0.362173,-0.114123,0.231388,0.039895,0.221328,0.278169


## Selecting the 50 Best Momentum Stocks

As before, we can identify the 50 best momentum stocks in our universe by sorting the DataFrame on the HQM Score column and dropping all but the top 50 entries.

In [13]:
hqm_dataframe.sort_values('HQM Score', ascending = False, inplace = True)
hqm_dataframe = hqm_dataframe[:50]
hqm_dataframe.reset_index(inplace=True, drop = True)
hqm_dataframe

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FTI,12.08,N/A,0.951092,0.985915,0.582931,0.993964,0.424143,0.995976,0.210545,0.891348,0.966801
1,GILD,83.82,N/A,0.283723,0.879276,0.358041,0.961771,0.300328,0.981891,0.271924,0.963783,0.94668
2,SLB,55.20,N/A,0.792346,0.971831,0.322222,0.949698,0.453026,0.997988,0.182735,0.837022,0.939135
3,ABMD,395.89,N/A,0.112858,0.756539,0.609640,0.997988,0.402402,0.991952,0.508589,1.0,0.93662
4,NOV,22.85,N/A,0.935807,0.981891,0.283712,0.915493,0.278323,0.977867,0.161150,0.762575,0.909457
5,COG,22.93,N/A,0.346451,0.909457,0.182607,0.808853,0.256733,0.971831,0.251728,0.94165,0.907948
6,RJF,122.04,N/A,0.265154,0.871227,0.351316,0.957746,0.124469,0.877264,0.216000,0.897384,0.900905
7,AFL,70.96,N/A,0.318957,0.89336,0.303808,0.937626,0.129172,0.885312,0.194562,0.861167,0.894366
8,AMP,338.80,N/A,0.120105,0.762575,0.316256,0.943662,0.163941,0.915493,0.253345,0.949698,0.892857
9,LVS,44.83,N/A,0.140356,0.784708,0.322199,0.947686,0.165885,0.917505,0.229039,0.919517,0.892354


## Calculating the Number of Shares to Buy

We'll use the portfolio_input function that we created earlier to accept our portfolio size. Then we will use similar logic in a for loop to calculate the number of shares to buy for each stock in our investment universe.

In [14]:
portfolio_input()

Enter the size of your portfolio: 1000000


In [15]:
position_size = float(portfolio_size)/len(hqm_dataframe.index)
for i in hqm_dataframe.index:
    hqm_dataframe.loc[i,'Number of Shares to Buy'] = math.floor(position_size/hqm_dataframe.loc[i,'Price'])
hqm_dataframe

C:\Users\bison\anaconda3\lib\site-packages\pandas\core\indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Six-Month Price Return,Six-Month Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,HQM Score
0,FTI,12.08,1655,0.951092,0.985915,0.582931,0.993964,0.424143,0.995976,0.210545,0.891348,0.966801
1,GILD,83.82,238,0.283723,0.879276,0.358041,0.961771,0.300328,0.981891,0.271924,0.963783,0.94668
2,SLB,55.20,362,0.792346,0.971831,0.322222,0.949698,0.453026,0.997988,0.182735,0.837022,0.939135
3,ABMD,395.89,50,0.112858,0.756539,0.609640,0.997988,0.402402,0.991952,0.508589,1.0,0.93662
4,NOV,22.85,875,0.935807,0.981891,0.283712,0.915493,0.278323,0.977867,0.161150,0.762575,0.909457
5,COG,22.93,872,0.346451,0.909457,0.182607,0.808853,0.256733,0.971831,0.251728,0.94165,0.907948
6,RJF,122.04,163,0.265154,0.871227,0.351316,0.957746,0.124469,0.877264,0.216000,0.897384,0.900905
7,AFL,70.96,281,0.318957,0.89336,0.303808,0.937626,0.129172,0.885312,0.194562,0.861167,0.894366
8,AMP,338.80,59,0.120105,0.762575,0.316256,0.943662,0.163941,0.915493,0.253345,0.949698,0.892857
9,LVS,44.83,446,0.140356,0.784708,0.322199,0.947686,0.165885,0.917505,0.229039,0.919517,0.892354


## Formatting Our Excel Output

We will be using the XlsxWriter library for Python to create nicely-formatted Excel files.

XlsxWriter is an excellent package and offers tons of customization. However, the tradeoff for this is that the library can seem very complicated to new users. Accordingly, this section will be fairly long because I want to do a good job of explaining how XlsxWriter works.

In [16]:
writer = pd.ExcelWriter('momentum_strategy.xlsx', engine='xlsxwriter')
hqm_dataframe.to_excel(writer, sheet_name = "Momentum Strategy", index = False)


## Creating Format Needed for .xlsx File

You'll recall from our first project that formats include colors, fonts, and also symbols like % and $. We'll need four main formats for our Excel document:

- String format for tickers
- XX.XX format for stock prices
- XX,XXX format for market capitalization
- Integer format for the number of shares to purchase

Since we already built our formats in the last section of this course, I've included them below for you. Run this code cell before proceeding.

In [17]:
background_color = '#0a0a23'
font_color = '#ffffff'

string_template = writer.book.add_format(
        {
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

dollar_template = writer.book.add_format(
        {
            'num_format':'$0.00',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

integer_template = writer.book.add_format(
        {
            'num_format':'0',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

percent_template = writer.book.add_format(
        {
            'num_format':'0.0%',
            'font_color': font_color,
            'bg_color': background_color,
            'border': 1
        }
    )

In [18]:
column_formats = {'A': ['Ticker', string_template],
                  'B': ['Price', dollar_template],
                  'C': ['Number of Shares to Buy', integer_template],
                  'D': ['One-Year Price Return', percent_template],
                  'E': ['One-Year Return Percentile', percent_template],
                  'F': ['Six-Month Price Return', percent_template],
                  'G': ['Six-Month Return Percentile', percent_template],
                  'H': ['Three-Month Price Return', percent_template],
                  'I': ['Three-Month Return Percentile', percent_template],
                  'J': ['One-Month Price Return', percent_template],
                  'K': ['One-Month Return Percentile', percent_template],
                  'L': ['HQM Score', percent_template]}

for column in column_formats.keys():
    writer.sheets['Momentum Strategy'].set_column(f'{column}:{column}', 25, column_formats[column][1])
    writer.sheets['Momentum Strategy'].write(f'{column}1', column_formats[column][0], column_formats[column][1])

## Saving Excel Output

In [19]:
writer.save()